# Imports

In [15]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests
import pandas as pd
import re

# Scraping Sustainalytics Ratings Data

## Configuring Selenium

In [2]:
# Set path to chromedriver
PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
# Define driver
driver = webdriver.Chrome(PATH)
# Define url
url= "https://www.sustainalytics.com/esg-ratings/"
# Maximize window
driver.maximize_window()
# Get website
driver.get(url)

## Creating DataFrame of Ratings

In [3]:
tickers = ['AAPL','AMZN']

In [4]:
# Creating dataframe for ratings data
ratings = pd.DataFrame(columns = ['company','risk_rating','risk_category','industry_group_position','overall_rank', 'rater'])

In [5]:
for ticker in tickers:
    # Find search bar
    search_bar = driver.find_element_by_id("searchInput")
    # Sleep for 5 seconds
    time.sleep(5)
    # Clear search bar
    search_bar.clear()
    # Enter ticker into search bar
    search_bar.send_keys(ticker)
    # Sleep for 5 seconds
    time.sleep(5)
    # Search ticker
    search_bar.send_keys(Keys.ENTER)
    # Find company in options list
    # Sleep for 3 seconds
    time.sleep(3)
    company = driver.find_element_by_class_name("companyName")
    # Click company for ratings
    company.click()

    # Getting page content
    content = driver.page_source.encode('utf-8').strip()
    # Getting page content in html
    soup = BeautifulSoup(content,"html.parser")
    # Getting company details
    company_details = soup.find(class_="row company-details d-flex")
    # Setting company name
    company = ticker
    # Getting risk ratings
    risk_rating = company_details.find(class_="col-xs-6 risk-rating-score").text
    # Getting risk category
    risk_category = company_details.find(class_="col-xs-6 risk-rating-assessment").text
    # Getting industry group position
    industry_group_position = (int(company_details.find(class_="industry-group-position").text) / int(company_details.find(class_="industry-group-positions-total").text)) * 100
    # Getting overall rank
    overall_rank = (int(company_details.find(class_="universe-position").text) / int(company_details.find(class_="universe-positions-total").text)) * 100
    # Appending new ratings to dataframe
    ratings = ratings.append({'company': company,
                              'risk_rating': risk_rating, 
                              'risk_category': risk_category, 
                              'industry_group_position': industry_group_position, 
                              'overall_rank': overall_rank
                              'rater': 'sustainalytics'},
                               ignore_index=True)
    # Sleep for 3 seconds
    time.sleep(3)
    # Go back to previous page
    driver.back()

    # Sleep for 3 seconds
    time.sleep(3)

In [6]:
ratings

,company,risk_rating,risk_category,industry_group_position,overall_rank
0,AAPL,16.6,Low Risk,28.597786,10.029828
1,AMZN,27.4,Medium Risk,99.779736,46.726324


# Scraping MSCI Ratings Data

In [7]:
# Set path to chromedriver
PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
# Define driver
driver = webdriver.Chrome(PATH)
# Define url
url= "https://www.msci.com/our-solutions/esg-investing/esg-ratings/esg-ratings-corporate-search-tool"
# Maximize window
driver.maximize_window()
# Get website
driver.get(url)

In [22]:
# Find search bar
search_bar = driver.find_element_by_id("_esgratingsprofile_keywords")
# Sleep for 5 secondsb
time.sleep(5)
# Clear search bar
search_bar.clear()
# Enter ticker into search bar
search_bar.send_keys("GOOG")
# Sleep for 5 seconds
time.sleep(5)
# Search ticker
search_bar.send_keys(Keys.ENTER)
# Find company in options list
# Sleep for 3 seconds
time.sleep(3)
company = driver.find_element_by_id("ui-id-1")
# Click company for ratings
company.click()

In [16]:
# Getting page content
content = driver.page_source.encode('utf-8').strip()
# Getting page content in html
soup = BeautifulSoup(content,"html.parser")
# Getting company details
company_details = soup.find(class_="ratingdata-outercircle esgratings-profile-header-yellow")
# Getting risk rating
risk_rating = ((str(company_details)).split('-'))[-1][:4].replace('"','')
risk_rating = re.sub(r"[^a-z]+", '', risk_rating)

In [17]:
risk_rating

'a'